## 1. Getting VizDoom up and running

In [4]:
# import VizDoom for game env
from vizdoom import *
# Import random for action sampling
import random
# Import time for sleeping
import time
# import numpy for identity matrix
import numpy as np

from matplotlib import pyplot as plt

## 2. Converting it to a Gym Environment

In [5]:
# Import environment base class from OpenAI Gym
from gymnasium import Env
# Import gym spaces
from gymnasium.spaces import Discrete, Box
# Import Opencv for greyscaling observations
import cv2

# Import environment checker
from stable_baselines3.common import env_checker
# Discrete(3).sample() returns a number from 0, 1, 2 -> used as index to select action
# Box(low=0, high=10, shape=(10,10)).sample() -> getting 10x10 array with low=0 and high=10

LEVEL = 'deadly_corridor'
DOOM_SKILL = 's1'

### Environment configuration for Reward Shaping 
#### Additional game variables needed for this level:
- DAMAGE_TAKEN (-)
- DAMAGECOUNT (+)
- SELECTED_WEAPON_AMMO (-)

In [6]:
# Create VizDoom OpenAI Gym Environment
class VizDoomGym(Env): 
    def __init__(self, render=False, config=f'VizDoom/scenarios/{LEVEL}_{DOOM_SKILL}.cfg'):
        """
        Function called when we start the env.
        """

        # Inherit from Env
        super().__init__()
        
        # Set up game
        self.game = DoomGame()
        self.game.load_config(config)
        

        # Whether we want to render the game 
        if render == False:
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)

        # Start the game
        self.game.init()
        
        # Create action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100, 160, 1), dtype=np.uint8)
        self.action_space = Discrete(7)

        # Game variables: HEALTH DAMAGE_TAKEN DAMAGECOUNT SELECTED_WEAPON_AMMO 
        ## We want the change in these variable values, rather than the PiT values
        self.damage_taken = 0
        self.damagecount = 0
        self.ammo = 52

    
    def step(self, action):
        """
        How we take a step in the environment.
        """

        # Specify action and take step
        actions = np.identity(7, dtype=np.uint8)
        # Movement rewards encapsulates predefined reward in the environment config
        movement_reward = self.game.make_action(actions[action], 4) # get action using index -> left, right, shoot

        reward = 0
        # Get all the other stuff we need to return 
        if self.game.get_state():  # if nothing is
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)  # Apply Grayscale
            # ammo = self.game.get_state().game_variables[0] 

            # Reward shaping
            game_variables = self.game.get_state().game_variables # get current PiT game variables
            health, damage_taken, damagecount, ammo = game_variables # unpack

            # calculate change in damage_taken, damagecount, ammo
            damage_taken_delta = -damage_taken + self.damage_taken # disincentivizng us to take damage
            self.damage_taken = damage_taken
            damagecount_delta = damagecount - self.damagecount # increments by +1: incentivizing more damagecounts (1 damagecount = 1 reward)
            self.damagecount = damagecount
            ammo_delta = ammo - self.ammo # increments by -1: disincentiving us to take shots that miss
                                          # damagecount and ammo will cancel each other out
            self.ammo = ammo

            # Pack everything into reward function (tuned weights)
            reward = movement_reward + damage_taken_delta*10 + damagecount_delta*200 + ammo_delta*5
            
            info = ammo
        # If we dont have anything turned from game.get_state
        else:
            # Return a numpy zero array
            state = np.zeros(self.observation_space.shape)
            # Return info (game variables) as zero
            info = 0

        info = {"info":info}
        done = self.game.is_episode_finished()
        truncated = False  # Assuming it's not truncated, modify if applicable
        
        return state, reward, done, truncated, info

    
    def render(self):
        """
        Define how to render the game environment.
        """
        pass

    
    def reset(self, seed=None):
        """
        Function for defining what happens when we start a new game.
        """
        if seed is not None:
            self.game.set_seed(seed)
            
        self.game.new_episode()
        state = self.game.get_state().screen_buffer  # Apply Grayscale

        return self.grayscale(state), {}

    
    def grayscale(self, observation):
        """
        Function to grayscale the game frame and resize it.
        observation: gameframe
        """
        # Change colour channels 
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)

        # Reduce image pixel size for faster training
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize,(100, 160,1))
        return state

    def close(self):
        """
        Call to close down the game.
        """
        self.game.close()


In [8]:
env = VizDoomGym(render=True)

Environment checker

In [9]:
env_checker.check_env(env)

In [10]:
env.close()

## 3. Setup Callback
Save model at different state of training

In [11]:
# Import os for file nav
import os
# Import callback class from sb3
from stable_baselines3.common.callbacks import BaseCallback

In [12]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        return True
                                      

In [13]:
CHECKPOINT_DIR = './train/train_corridor'
LOG_DIR = './logs/log_corridor'

In [14]:
# Create instance of callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR) 
# after every 10000 steps of training the model, weights are saved for the pytorch agent

## 5. Train Model using Curriculum


In [15]:
# Import PPO for training
from stable_baselines3 import PPO

In [16]:
# Non rendered environment
DOOM_SKILL = 's1'
env = VizDoomGym(config=f'VizDoom/scenarios/{LEVEL}_{DOOM_SKILL}.cfg')

In [17]:
# n_steps: How many steps/frames the agent is going to take and store in the buffer 
# before run through training of actor and critique
# ideally not too close to end of game (300) but somewhere close
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, learning_rate=0.00001, n_steps=8192, clip_range=.1, gamma=.95, gae_lambda=.9) 
# n_step = batchsize
# clip_range clips the gradient
# gae_lambda smoothing factor

# if ep_len drop and ep_rew shoots up likely because agent is running straight forward
# ep_len ideally should keep going down but not too fast, ep_rew should keep going up but not too fast (else unstable training)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [18]:
model.learn(total_timesteps=400000, callback=callback)

Logging to ./logs/log_corridor/PPO_1
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 183      |
|    ep_rew_mean     | 82.5     |
| time/              |          |
|    fps             | 36       |
|    iterations      | 1        |
|    time_elapsed    | 227      |
|    total_timesteps | 8192     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 186          |
|    ep_rew_mean          | 127          |
| time/                   |              |
|    fps                  | 25           |
|    iterations           | 2            |
|    time_elapsed         | 634          |
|    total_timesteps      | 16384        |
| train/                  |              |
|    approx_kl            | 0.0024399245 |
|    clip_fraction        | 0.133        |
|    clip_range           | 0.1          |
|    entropy_loss         | -1.94        |
|    explained_varian

KeyboardInterrupt: 

#### Load saved best model and apply Curriculum Learning (S2 - S5)

In [ ]:
model.load('./train/train_corridor/best_model_260000.zip')

In [ ]:
# Non rendered environment for S2
DOOM_SKILL = 's2'
env = VizDoomGym(config=f'VizDoom/scenarios/{LEVEL}_{DOOM_SKILL}.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

In [ ]:
# Non rendered environment for S3
DOOM_SKILL = 's3'
env = VizDoomGym(config=f'VizDoom/scenarios/{LEVEL}_{DOOM_SKILL}.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

In [ ]:
# Non rendered environment for S4
DOOM_SKILL = 's4'
env = VizDoomGym(config=f'VizDoom/scenarios/{LEVEL}_{DOOM_SKILL}.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

In [ ]:
# Non rendered environment for S5
DOOM_SKILL = 's5'
env = VizDoomGym(config=f'VizDoom/scenarios/{LEVEL}_{DOOM_SKILL}.cfg')
model.set_env(env)
model.learn(total_timesteps=400000, callback=callback)

## 5. Test Model


In [ ]:
# Import eval policy to test agent
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
# Reload model from disc
model = PPO.load('./train/train_defend/best_model_70000')

In [ ]:
# Create rendered envrironment
env = VizDoomGym(render=True)

In [ ]:
# Evaluate mean reward for 10 games
mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=10)

/Users/csqh/anaconda3/envs/pythonlab/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [ ]:
mean_reward

6.0

In [ ]:

for episode in range(5):
    total_reward = 0
    obs = env.reset()[0]
    done = False
    while not done:
        action, _ = model.predict(obs) # Use model to predict what action to take
        obs, reward, done, _, info = env.step(action) # take the predicted action
        time.sleep(0.25)
        total_reward += reward
    print('Total Reward for episode {} is {}'.format(total_reward, episode))
    time.sleep(2)

Total Reward for episode 5.0 is 0
Total Reward for episode 12.0 is 1
Total Reward for episode 6.0 is 2
Total Reward for episode 10.0 is 3
Total Reward for episode 10.0 is 4


In [ ]:
env.close()